### Задание №1.
Можно ли отобрать наиболее значимые признаки с помощью PCA? Ответ объясните.

__Ответ:__ Отобрать наиболее значимые признаки из первоначальных с помощью PCA невозможно, так как метод главных компонент в результате преобразования образует совершенно другую матрицу "объект-признак" по значениям не связанную с первоначальной.
___

### Задание №2.
Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd(). Применить к данным на уроке и сравнить ответы.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
def std_scale(x):
    x_ = x.copy()
    return (x_ - x_.mean(0)) / x_.std(0)

При сингулярном разложении получаем:
$$X=UDV^T,$$
где, среди прочих, $D$ - это диагональная матрица, на главной диагонали которой располагаются квадратные корни собственных чисел матрицы $X^TX$ в порядке убывания значений, а $V$ - ортогональная матрица, столбцы которой являются собственными векторами матрицы $X^TX$, соответствующими собственным числам в порядке убывания значений.

Тогда для нахождения матрицы весов для метода PCA просто берем первые n столбцов матрицы $X^TX$ и умножаем ее на первоначальную матрицу "объект-признак" справа.

In [3]:
def pca(X, n=None, verbose=False):
    
    # Выполним стандартизацию признаков
    X_st = std_scale(X)
    
    # Найдем собственные значения и векторы из сингулярного разложения
    eig_values, eig_vectors =np.linalg.svd(X_st)[1] ** 2, np.linalg.svd(X_st)[2]
    
    # Посчитаем долю дисперсии, описываемой каждой компонентой
    var_exp = eig_values * 100 / eig_values.sum()
    cum_var_exp = var_exp.cumsum()
    
    if verbose:
        print(f'Собственные значения:\n{eig_values}\n'
              f'И соответствующие им собственные векторы:\n{eig_vectors.T}\n'
              f'Доля дисперсии, описываемой каждой из компонент (%):\n{var_exp}\n'
              f'Кумулятивная доля дисперсии по компонентам (%):\n{cum_var_exp}\n')
    
    return X_st.dot(eig_vectors[:n].T) if n else X_st.dot(eig_vectors.T)

In [4]:
iris = datasets.load_iris()
X, y = iris.data, iris.target

In [5]:
pca(X, 2, verbose=True)

Собственные значения:
[437.77467248 137.10457072  22.01353134   3.10722546]
И соответствующие им собственные векторы:
[[ 0.52106591 -0.37741762  0.71956635  0.26128628]
 [-0.26934744 -0.92329566 -0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161 -0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199 -0.63427274  0.52359713]]
Доля дисперсии, описываемой каждой из компонент (%):
[72.96244541 22.85076179  3.66892189  0.51787091]
Кумулятивная доля дисперсии по компонентам (%):
[ 72.96244541  95.8132072   99.48212909 100.        ]



array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356],
       [-2.1663101 , -1.04369065],
       [-2.32613087, -0.13307834],
       [-2.2184509 ,  0.72867617],
       [-2.6331007 ,  0.96150673],
       [-2.1987406 , -1.86005711],
       [-2.26221453, -2.68628449],
       [-2.2075877 , -1.48360936],
       [-2.19034951, -0.48883832],
       [-1.898572  , -1.40501879],
       [-2.34336905, -1.12784938],
       [-1.914323  , -0.40885571],
       [-2.20701284, -0.92412143],
       [-2.7743447 , -0.45834367],
       [-1.81866953, -0.08555853],
       [-2.22716331, -0.13725446],
       [-1.95184633,  0.62561859],
       [-2.05115137, -0.24216355],
       [-2.16857717, -0.52714953],
       [-2.13956345,

___

### Задание №3.
Обучить любую модель классификации (из рассмотренных в курсе) на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

В качестве модели классификации возьмем одно дерево решений:

In [6]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index
        self.t = t
        self.true_branch = true_branch
        self.false_branch = false_branch


class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.classes = self.get_classes()
        self.prediction = self.predict()
    
    def get_classes(self):
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        return classes
        
    def predict(self):    
        prediction = max(self.classes, key=self.classes.get)
        return prediction


def get_inform_index(labels, type_index='gini'):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    if type == 'gini':
        impurity = 1
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p ** 2
    else:
        impurity = 0
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p * np.log2(p) 
        
    return impurity


def quality(left_labels, right_labels, type_index, current_index):

    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_index - p * get_inform_index(left_labels, type_index) - (1 - p) * get_inform_index(right_labels, type_index)


def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels


def find_best_split(data, labels, min_leaf, inform_index):
    current_index = get_inform_index(labels, inform_index)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, inform_index, current_index)
            
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index


def build_tree(data, labels, min_leaf=5, max_depth=None, inform_index='gini'):

    quality, t, index = find_best_split(data, labels, min_leaf, inform_index)

    if quality == 0 or max_depth == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    if max_depth:
        true_branch = build_tree(true_data, true_labels, min_leaf, max_depth - 1)
        false_branch = build_tree(false_data, false_labels, min_leaf, max_depth - 1)
    else:
        true_branch = build_tree(true_data, true_labels, min_leaf)
        false_branch = build_tree(false_data, false_labels, min_leaf)
        
    return Node(index, t, true_branch, false_branch)


def classify_object(obj, node):

    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)


def predict(data, tree):
    
    preds = []
    for obj in data:
        prediction = classify_object(obj, tree)
        preds.append(prediction)
    return np.array(preds)


def accuracy_metric(real, pred):
    return np.sum(real == pred) / real.shape[0]

Понизим размерность данных методом главных компонент до 2 признаков:

In [7]:
X_pca = pca(X, 2)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=23)

Построим деревеья для обоих алгоритмов с оптимальными гиперпараметрами, обучим их и сделаем предсказания на тестовых выборках:

In [9]:
tree = build_tree(X_train, y_train, min_leaf=5, max_depth=None)
tree_pca = build_tree(X_train_pca, y_train_pca, min_leaf=1, max_depth=7)

In [10]:
train_answers = predict(X_train, tree)
test_answers = predict(X_test, tree)
train_answers_pca = predict(X_train_pca, tree_pca)
test_answers_pca = predict(X_test_pca, tree_pca)

Метрики accuracy для обучающей и тестовой выборок соответственно:

In [11]:
accuracy_metric(y_train, train_answers)

0.9619047619047619

In [12]:
accuracy_metric(y_test, test_answers)

0.9555555555555556

Метрики accuracy для обучающей и тестовой выборок соответственно c понижением размерности методом главных компонент:

In [13]:
accuracy_metric(y_train_pca, train_answers_pca)

0.9904761904761905

In [14]:
accuracy_metric(y_test_pca, test_answers_pca)

0.9555555555555556

Как видим, в частности, для деревьев решений с понижением размерности данных методом главных копмонент можно подобрать оптимальные гиперпараметры, при которых алгоритм будет выдавать не худшие результаты по сравенению с алгоритмами, обученными на полных данных.